In [1]:
import os, sys
# PWD = os.getenv('PWD')
PWD = os.path.abspath('.')

PROJ_MISSING_MSG = """Set an enviroment variable:\n
`DJANGO_PROJECT=your_project_name`\n
or call:\n
`init_django(your_project_name)`
"""

def init_django(project_name=None):
    os.chdir(PWD)
    project_name = project_name or os.environ.get('DJANGO_PROJECT') or None
    if project_name == None:
        raise Exception(PROJ_MISSING_MSG)
    sys.path.insert(0, os.getenv('PWD'))
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    import django
    django.setup()

In [2]:
init_django('bae_bi') #project name

In [7]:
from board.models import Category, Store, Delivery_info, Menu, Promotion, User, Review
import json

In [6]:
# Category 저장, img path 추가 필요

categories = []
categories.append(Category(name="치킨", url = "chicken"))
categories.append(Category(name="중국집", url = "chinese"))
categories.append(Category(name="일식", url = "japanese"))
categories.append(Category(name="족발", url = "jokbal"))
categories.append(Category(name="한식", url = "korean"))
categories.append(Category(name="피자", url = "pizza"))
Category.objects.bulk_create(categories)

[<Category: Category object (None)>,
 <Category: Category object (None)>,
 <Category: Category object (None)>,
 <Category: Category object (None)>,
 <Category: Category object (None)>,
 <Category: Category object (None)>]

In [11]:
stores = []
menus = []
promotions = []
users = []
reviews = []
delivery_infos = []

with open(r'../data/chicken_infos.json', 'r') as f:
    info = json.load(f)

ck_id = Category.objects.filter(name="치킨")[0].id
s_id = 0
for name in info.keys():
    # store 추가
    store = {'name':'', 'address': '', 'thumbnail_path':'', 'category_id':'', 'avg_rate':0}
    store['name'] = name
    store['address'] = ''
    store['thumbnail_path'] = info[name]['logo']
    store['category_id'] = ck_id
    store['avg_rate'] = 0
    
    stores.append(store)
    
    # menu 추가
    store = {'name':'', 'address': '', 'thumbnail_path':'', 'category_id':'', 'avg_rate':0}

dict_keys(['굽네치킨&피자-동판교점', '맘스터치-판교밸리고등점', '굽네치킨&피자-이매점', '롯데리아-판교DT점', '굽네치킨&피자-성남동점', '헤라s키친', '맘스터치-서판교점', '비트김밥&닭강정-서현점', '닭터박강정떡볶이-분당서현점', '포포', '청년다방-판교테크노밸리점', '노모어피자-야탑점', '조나단돈까스'])

In [18]:
Category.objects.order_by('-id')[0].id

6

In [20]:
Store.objects.order_by('-id')

<QuerySet []>